In [1]:
'''
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Подготовка модели и токенизатора
model_id = "deepvk/RuModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)
model.to('cuda')
model = model.eval()

# Подготовка входного текста
text = ["Лимончелло это настойка из лимона."]
inputs = tokenizer(text, return_tensors="pt")
inputs = {key: value.to('cuda') for key, value in inputs.items()}

outputs = model(**inputs)

# Получение эмбеддингов с применением mean pooling
# Для получения скрытых состояний передаем output_hidden_states=True
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)
    # Последний скрытый слой (размер [batch_size, sequence_length, hidden_dim])
    hidden_states = outputs.hidden_states[-1]

attention_mask = inputs["attention_mask"].squeeze(0)

def mean_pooling(hidden_states, attention_mask):
    """
    Вычисляет среднее значение эмбеддингов токенов с учетом attention mask.
    """
    # Расширяем attention mask до размера эмбеддингов и приводим к float
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()[0:hidden_states.size(0),]
    # Суммируем эмбеддинги активных токенов (без паддинга)
    sum_embeddings = torch.sum(hidden_states[0:hidden_states.size(0),] * input_mask_expanded, dim=0)
    # Считаем число активных токенов (чтобы избежать деления на ноль ставим минимум)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=0), min=1e-9)
    return sum_embeddings / sum_mask

sentence_embedding = mean_pooling(hidden_states, attention_mask)
print(sentence_embedding.shape)
print("Sentence embedding:", sentence_embedding)
'''

'\nimport torch\nfrom transformers import AutoTokenizer, AutoModelForMaskedLM\n\n# Подготовка модели и токенизатора\nmodel_id = "deepvk/RuModernBERT-base"\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForMaskedLM.from_pretrained(model_id)\nmodel.to(\'cuda\')\nmodel = model.eval()\n\n# Подготовка входного текста\ntext = ["Лимончелло это настойка из лимона."]\ninputs = tokenizer(text, return_tensors="pt")\ninputs = {key: value.to(\'cuda\') for key, value in inputs.items()}\n\noutputs = model(**inputs)\n\n# Получение эмбеддингов с применением mean pooling\n# Для получения скрытых состояний передаем output_hidden_states=True\nwith torch.no_grad():\n    outputs = model(**inputs, output_hidden_states=True)\n    # Последний скрытый слой (размер [batch_size, sequence_length, hidden_dim])\n    hidden_states = outputs.hidden_states[-1]\n\nattention_mask = inputs["attention_mask"].squeeze(0)\n\ndef mean_pooling(hidden_states, attention_mask):\n    """\n    Вычисляет средн

In [2]:
'''
import re
import string

# Подготовим экранированную строку с допустимыми знаками препинания
punct = re.escape(string.punctuation)

# Пример исходного текста
txt = """Это обычный текст.
РУССКИЕ ЗАГЛАВНЫЕ!
РУССКИЕ ЗАГЛАВНЫЕ 123
Это другой текст.
ЭТО ПОЛНОСТЬЮ КАПС ЛОК.
Еще один текст.
"""


# Регулярное выражение для удаления блоков:
# (?ms)            - включаем режим многострочный и dotall (точка захватывает переводы строк)
# ^([А-ЯЁ\s{punct}]+)\r?\n  - находим строку, которая целиком состоит из русских заглавных букв, пробелов и знаков препинания, включая завершающий перевод строки.
# (?:.*\r?\n)+?    - затем нежадно (non-greedy) захватываем одну или несколько строк любого содержания (это – промежуточный текст).
# [А-ЯЁ\s{punct}]+(?:\r?\n|$) - заканчиваем блок строкой, которая опять состоит только из русских заглавных букв (с завершающим переводом строки или концом текста).

punct = re.escape(string.punctuation)
pattern_caps = rf'(?m)^[А-ЯЁ0-9\s{punct}]+$(?:\r?\n)?'

# Выполняем замену – удаляем найденные блоки.
new_txt = re.sub(pattern_caps, '', txt)

print("Текст после удаления блоков (CAPS LOCK + промежуточный текст):")
print(new_txt)
'''


'\nimport re\nimport string\n\n# Подготовим экранированную строку с допустимыми знаками препинания\npunct = re.escape(string.punctuation)\n\n# Пример исходного текста\ntxt = """Это обычный текст.\nРУССКИЕ ЗАГЛАВНЫЕ!\nРУССКИЕ ЗАГЛАВНЫЕ 123\nЭто другой текст.\nЭТО ПОЛНОСТЬЮ КАПС ЛОК.\nЕще один текст.\n"""\n\n\n# Регулярное выражение для удаления блоков:\n# (?ms)            - включаем режим многострочный и dotall (точка захватывает переводы строк)\n# ^([А-ЯЁ\\s{punct}]+)\r?\n  - находим строку, которая целиком состоит из русских заглавных букв, пробелов и знаков препинания, включая завершающий перевод строки.\n# (?:.*\r?\n)+?    - затем нежадно (non-greedy) захватываем одну или несколько строк любого содержания (это – промежуточный текст).\n# [А-ЯЁ\\s{punct}]+(?:\r?\n|$) - заканчиваем блок строкой, которая опять состоит только из русских заглавных букв (с завершающим переводом строки или концом текста).\n\npunct = re.escape(string.punctuation)\npattern_caps = rf\'(?m)^[А-ЯЁ0-9\\s{punct}]+

In [3]:
#%pip install faiss-cpu

In [1]:
import re
import logging
from split_util import RecursiveCharacterTextSplitter
from split_util import Doc
import pandas as pd
import numpy as np
import os
import re
import pickle as pkl
from datetime import datetime
import inspect
import configparser
from tqdm.notebook import tqdm
from dataclasses import dataclass
import random
from rank_bm25 import BM25Okapi

from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
import torch
import torch.nn as nn

from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, AutoModelForCausalLM
import string

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-03-24 13:12:28.891051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-24 13:12:29.669592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Reserved memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Allocated memory: 0.00 MB
Reserved memory: 0.00 MB


In [3]:
!nvidia-smi

Mon Mar 24 13:12:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.03             Driver Version: 535.216.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:8C:00.0 Off |                    0 |
| N/A   33C    P0              77W / 500W |      9MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
fl_ls = [x for x in os.listdir("./input") if os.path.splitext(x)[1] == ".txt" ]
print(fl_ls)

['law_4.txt', 'law_8.txt', 'law_5.txt', 'law_3.txt', 'law_7.txt', 'law_6.txt', 'law_9.txt', 'law_10.txt', 'law_12.txt', 'law_11.txt', 'law_14.txt', 'law_13.txt', 'law_16.txt', 'law_18.txt', 'law_15.txt', 'law_17.txt', 'law_2.txt', 'law_1.txt']


In [5]:
class Config():
    """
        класс загрузки данных из config
    """
    def __init__(self):
        config = configparser.ConfigParser()
        config.read('config_10.ini')
        #config.read('config_lc_bge.ini')
        #config.read('config_lc_bert.ini')
        self.temperature_qst_gen = float(config['temperature']['temperature_qst_gen'])
        self.temperature_fact_check = float(config['temperature']['temperature_fact_check'])
        self.temperature_response_gen = float(config['temperature']['temperature_response_gen'])

        self.prompt_template_qst = config['prompts']['prompt_template_qst']
        self.prompt_template_check = config['prompts']['prompt_template_check']
        self.prompt_template_response = config['prompts']['prompt_template_response']
        self.prompt_template_qst_response = config['prompts']['prompt_template_qst_response']
        self.system_prompt = config['prompts']['system_prompt']
        self.prompt_template_response_ext = config['prompts']['prompt_template_response_ext']
        
        self.base_path = config['path']['base_path']

        self.reranker_device = config['reranker']['reranker_device']
        self.doc_reranker_name = config['reranker']['doc_reranker_name']

        self.doc_emb_model_name = config['embedding']['doc_emb_model_name']
        self.emb_model_device = config['embedding']['emb_model_device']  

        #self.late_chunk_model_name = config['late_chunk_embedding']['late_chunk_model_name']
        #self.emb_model_device = config['late_chunk_embedding']['emb_model_device']
        
        self.max_seq_len = config['late_chunk_embedding']['max_seq_len']

    def reload(self):
        config = configparser.ConfigParser()
        config.read('config_10.ini')
        #config.read('config_lc_bge.ini')
        #config.read('config_lc_bert.ini')
        self.temperature_qst_gen = float(config['temperature']['temperature_qst_gen'])
        self.temperature_fact_check = float(config['temperature']['temperature_fact_check'])
        self.temperature_response_gen = float(config['temperature']['temperature_response_gen'])

        self.prompt_template_qst = config['prompts']['prompt_template_qst']
        self.prompt_template_check = config['prompts']['prompt_template_check']
        self.prompt_template_response = config['prompts']['prompt_template_response']
        self.prompt_template_qst_response = config['prompts']['prompt_template_qst_response']
        self.system_prompt = config['prompts']['system_prompt']
        self.prompt_template_response_ext = config['prompts']['prompt_template_response_ext']

        self.base_path = config['path']['base_path']

        self.reranker_device = config['reranker']['reranker_device']
        self.doc_reranker_name = config['reranker']['doc_reranker_name']

        self.doc_emb_model_name = config['embedding']['doc_emb_model_name']
        self.emb_model_device = config['embedding']['emb_model_device']    
    
        #self.late_chunk_model_name = config['late_chunk_embedding']['late_chunk_model_name']
        #self.emb_model_device = config['late_chunk_embedding']['emb_model_device']
        
        self.max_seq_len = config['late_chunk_embedding']['max_seq_len']


In [6]:
#from rag_lc_bert import SearchBase
#from rag_lc_bert import Doc_Base
#from rag_lc_transformers import SearchBase
#from rag_lc_transformers import Doc_Base
from rag_transformers import SearchBase
from rag_transformers import Doc_Base

In [8]:
#base_tokenizer = AutoTokenizer.from_pretrained("deepvk/RuModernBERT-base")
#base_tokenizer.save_pretrained("./rumodernbert_finetuned")
#base_tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
#base_tokenizer.save_pretrained("./bgem3_finetuned")

('./bgem3_finetuned/tokenizer_config.json',
 './bgem3_finetuned/special_tokens_map.json',
 './bgem3_finetuned/sentencepiece.bpe.model',
 './bgem3_finetuned/added_tokens.json',
 './bgem3_finetuned/tokenizer.json')

In [7]:
#doc_base = Doc_Base(config=Config(), name='doc_base_test', path='./data')
doc_base = Doc_Base(name='doc_base_test', path='./data')
doc_base.fill(doc_path='./input')
doc_base.save()
doc_base.load()

In [12]:
#doc_base.base[1000]

In [13]:
#doc_base.base[1]

In [14]:
#doc_base.base[2]

In [15]:
#doc_base.base[3]

In [16]:
#doc_base.base[4]

In [9]:
search_base = SearchBase(config=Config())
search_base.fill(doc_base)
search_base.load()

Some weights of the model checkpoint at ./bgem3_finetuned were not used when initializing XLMRobertaModel: ['encoder.embeddings.LayerNorm.bias', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.word_embeddings.weight', 'encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.encoder.layer.0.attention.output.dense.bias', 'encoder.encoder.layer.0.attention.output.dense.weight', 'encoder.encoder.layer.0.attention.self.key.bias', 'encoder.encoder.layer.0.attention.self.key.weight', 'encoder.encoder.layer.0.attention.self.query.bias', 'encoder.encoder.layer.0.attention.self.query.weight', 'encoder.encoder.layer.0.attention.self.value.bias', 'encoder.encoder.layer.0.attention.self.value.weight', 'encoder.encoder.layer.0.intermediate.dense.bias', 'encoder.encoder.layer.0.intermediate.dense.weight', 'encoder.encode

In [18]:
#qst = "Здравствуйте. 07.03.2022 был открыт вклад в долларах США до 03.09.2022. 03.09.2022 было получено 10 000 долларов наличными и открыт новый вклад до 09.03.2023, затем открыт новый вклад до 15.06.2023, далее открыт вклад до 14.08.2025. Т.е. изначально валюта поступила в банк до 09.09.2022. Вопрос: по какому курсу в рубля (ЦБ или внутреннему курсу) банк должен вернуть средства с валютного вклада?"
#vector_search_res = search_base.search_doc_late(qst, doc_base)

In [ ]:
#for elem in vector_search_res:
#    print(elem)
#    print('\n')

In [ ]:
import re
import pandas as pd
'''
queries = [
    'Здравствуйте. 07.03.2022 был открыт вклад в долларах США до 03.09.2022. 03.09.2022 было получено 10 000 долларов наличными и открыт новый вклад до 09.03.2023, затем открыт новый вклад до 15.06.2023, далее открыт вклад до 14.08.2025. Т.е. изначально валюта поступила в банк до 09.09.2022. Вопрос: по какому курсу в рубля (ЦБ или внутреннему курсу) банк должен вернуть средства с валютного вклада?',
    'Здравствуйте! Разъясните, пожалуйста, позицию ЦБ РФ по вопросу возникших юридических проблем с открытыми валютными вкладами. Вопрос 1 :  могу ли я подарить валютный вклад, открытый до 09.03.22 и валюту, находящуюся на счете до 01.09.22 своему совершеннолетнему ребенку и каким образом ребенок сможет воспользоваться вкладом : получить разрешенные 10 тысяч долларов? или вклад и валюта счета обязательно конвертируются в рубли? Может ли он продолжить хранить валюту на счете? Вопрос 2 : в связи с необходимостью оформления наследства : каким образом наследуется валютный вклад , открытый до 09.03.2022 г.Какие действия можно с ним производить в случае одного наследника и в случае двух наследников? К сожалению, в РСХБ не смогли однозначно ответить на данные вопросы.',
    'Добрый день. Прошу дать пояснения по следующей ситуации. В тексте указа президента России «О дополнительных временных мерах экономического характера по обеспечению финансовой стабильности Российской Федерации» от 1 марта 2022 (далее – Указ) говориться о том, что я как физическое лицо не могу пополнять свои иностранные счета в валюте, но тем временем Вы (ЦБ РФ) увеличили лимит на переводы из РФ на счета, открытые за границей с 10 000 USD до 50 000 USD. Так же я имею права пользоваться услугами по переводу денежных средств без открытия банковского счета, предоставленными российскими финансовыми организациями. В виду трудностей интерпретации большого количества противоречий информации прошу дать пояснения по следующим вопросам: 1. Какие именно операции пополнения попадают под действие данного указа? 2. Могу ли я осуществлять переводы в валюте с использованием системы SWIFT со своих счетов открытых в Российской федерации и/или заграничных счетов на счета, открытые за границей? (Если можно, то в какие страны переводы под запретом?) 3. Могу ли я принимать переводы, осуществляемые без открытия банковского счета напрямую на счета в иностранных банках в случаях: а) если отправителем являюсь я сам б) отправитель – другое физическое лицо. Есть ли у этого метода ограничения, которые необходимо учитывать? 4. Распространяются ли ограниченные данного указа и/или других законодательных актов на валютных резидентов, которые более 183 дней находятся за пределами РФ в отношении совершения переводов со своих/на свои иностранные счета в валюте и совершения иных действия по этим счетам, включая пополнение иностранных счетов различными способами, внесение надличных на свои иностранные счета, переводы/пополнения третьими лицами своих иностранных счетов и иные возможные операции. 5. Каким правовым статусом обладают указы Президента РФ в части финансового и валютного регулирования? Как называются нарушения подобных указов? Насколько в отношении вышеобозначенного Указа и других указов президента РФ распространяется КоАП РФ Статья 15.25. (Нарушение валютного законодательства Российской Федерации и актов органов валютного регулировани и другие меры воздействия)? В статье 5 173-ФЗ «О валютном регулировании и валютном контроле» говорится о том, что «Органами валютного регулирования в Российской федерации являются Центральный банк Российской Федерации и Правительство Российской Федерации.». Как видно, президент РФ в этот список не входит, а значит за нарушения Указа не могут применяться санкции, применяемые за нарушение валютного законодательства.',
    'Добрый день! Интересует вопрос касательно использования денежных счетов со счета типа С, открытого в банке нерезиденту 1) согласно решению СД ЦБ с банковского счета типа «С» могут списываться денежные средства в целях переводов на покупку облигаций федерального займа, размещаемых Минфином России на аукционах, а также при обращении таких облигаций. Однако в настоящее время банки отказываются осуществлять данные операции в связи с тем, что нет порядка использования денежных средств. Когда планируется опубликовать данный порядок, чтобы осуществить разрешенные операции со счета типа С? 2) Предусмотрена ли выдача разрешений нерезидентам на использование денежных средств, хранящихся на счетах типа С, на иные не указанные в решении СД цели (например, на конвертацию в доллары США)?',
    '03.10.2025 в 08-31 банк списал с моего счета денежные средства в размере 4557,71 руб. Я сразу позвонила им и сообщила, чтобы деньги не перечисляли так как арест со счета снят и есть постановление суда. Следом я пошла в отделение банка города Пензы и отнесла постановление и попросила вернуть списанную сумму, на что мне пообещали вернуть в течении 3 дней. По истечению 3 х дней, сумма так и не вернулась я позвонила и составила жалобу. прошу Вас провести проверку действий банка и вернуть списанную сумму в размере 4557, 71 руб. в связи с незаконными действиями взыскателя и банка.',
    'На счета ООО "Вот это да" наложено несколько арестов: 1) 30 октября 2024 г. судьей городского суда вынесено Постановление об аресте счетов ООО "Вот это да" сроком до 13 ноября 2024 г. На сегодняшний день, а именно, 15.11.2024 г. арест со счетов не снят банком. Сотрудник ООО "Вот это да" оставлял обращение № 2024-1114-000 от 14.11.2024 г. в банке с просьбой пояснить, почему аресты до сих пор не сняты с связи с истечением срока наложения ареста, а именно -13.11.2024 г. На данное обращение сотрудник банка ответил, что Постановление еще в обработке и соответственно не могут дать ответ. Также, сотрудник банка начал утверждать, что это Постановление отправлено на подтверждение наличия данного ареста в ФССП - тем самым не разобравшись в том, что арест наложен городским судом, а не приставами. Таким образом, в организации арестованы все денежные средства, нет возможности выплачивать заработную плату сотрудникам организации, оплачивать работы/ услуги/материалы контрагентам, что приводит к срыву сроков контрактов, блокированию работы организации. 2) 3 октября 2023 г. Апелляционным постановлением судья областного суда установил срок арестов со счетов ООО "Вот это да" в пределах срока предварительно расследования уголовного дела, 23.01.2024 г. вынесено Постановление о прекращении уголовного дела № 0000, но по настоящее время арест со счетов не снят. Просьба в кратчайшие сроки принять меры по снятию арестов со счетов организации.',
    'Сообщаю Вам, что 17.09.2024 банк наложил арест на сумму 25 087,58 на мои счета и счета моей мамы и взыскал с нас сумму в размере 9 437,52, согласно судебного приказа от 04.07.2024. 18.09.2024 мы обратились в судебный участок и мировым судьей данного участка нам было выдано определение об отмене судебного приказа, которое в этот же день было передано в отделение банка. 19.09.2024 г. выяснилось, что при подаче сведений, суд допустил ошибку и неверно указал номер судебного приказа, что подтверждено и прописано в определении, а именно, изучив материалы дела, мировой судья пришел к выводу, что в тексте судебного приказа от 04.07.2024г. номер дела указан неверно, т.е. мировым судьей допущена описка. В отделение банка было передано два документа: 1) Определение об отмене судебного приказ от 18.09.2024 г., в котором судья определил отменить и отозвать с исполнения судебный приказ от 04.07.2024; 2) Определение от 18.09.2024 г., в котором судья определил исправить описку, допущенную в тексте судебного приказа от 18.09.2024 г. На основании вышеизложенного прошу в кротчайшие сроки разобраться в сложившейся ситуации, т.к. денежные средства списываются на основании отмененного документа, соответственно незаконно. Обращаю Ваше внимание, что все оригиналы документов переданы в отделение банка, находящемся по адресу. Прошу принять во внимание, что я возражаю на передачу моего заявления в поднадзорные органы.',
    'Здравствуйте ! Согласно графика платежей по ипотечному кредиту в дату 31.07.2020 я должен был внести на текущий счет в банке денежные средства в сумме 17 071,34 руб. Но в связи с ковидными ограничениями, банк изменил время работы офиса и в дату 31.07.2020 я не внес данную сумму. Банк 31.07.2020 в целях погашения ссуды списал остаток денежных средств с текущего счета в сумме 105,43 руб. 01.08.2020 я исполнил свои обязательства по ипотечному кредиту и внес на текущий счет сумму 17 100 руб. и в этот день банк списал с текущего счета сумму просроченной ссуды 4 521,63 руб. и просроченные проценты в сумме 12 444,08 руб. Соответственно количество просроченных дней составляет – 1 (Один день). Согласно кредитного отчета бки банк предоставил сведения, что у меня просроченная задолженность по состоянию на 31.08.2020 составляла 29 дней. 12.07.2024 я обратился в банк с просьбой выполнить корректировку по просроченной задолженности, на что банк ответил, что – «информация в бюро кредитных историй со стороны банка передана корректно». Обращаюсь в Банк России с просьбой разобраться в данной ситуации и обязать банк внести корректные сведения в бки.',
    'Я закрыла полностью кредит, но банк до сих пор не передал информацию в бки что нет долга, отображается 419р48копеек. Хотя мне справку дали что всё закрыто, с ареста авто сняли. Я по их вине не могу теперь ни чего сделать раз отображается долг в кредитной истории, хотя всю жизнь платила не было ни долга ни просрочек',
    'Добрый вечер. 08.11.2024 года подавал заявку в банк на ипотеку с господдержкой для семей с детьми на сайте, указав сумму первого взноса 21 % при стоимости жилья 12 500 000 рублей . В этот же день 08.11.2024 года мне пришел отказ от банка , после этого я зашел в личный кабинет бюро кредитных историй и увидел, что банк указал в заявке сумму 50 000 000 рублей. Позвонил в банк 08.11.2024 года и зарегистрировал обращение, что не согласен с суммой которую банк указала в заявке на семейную ипотеку с господдержкой в размере 50 000 000 рублей. 12.11.2024 я позвонил в банк, чтобы узнать статус моего обращения и мне сказали написать заявление и отправить его на электронный адрес. 28.11.2024 мне пришел ответ по моему заявлению от банка на мой электронный адрес. В ответе банк указали, что автоматически установили максимальную сумму заявки по ипотеке в размере 50 000 000 рублей тем самым испортив мой ПКР в БКИ на 200 балов. Автоматическое проставление суммы в размере 50 000 000 рублей нарушает мои права как субъекта кредитной истории. Вместе с этим хочу отметить, что при подаче заявки на Ипотеку мною была проставлена сумма стоимости жилья 12 500 000 рублей c первым взносом 21% ( так как лимит на ипотеку с господдержкой для семей с детьми по ставке 6% составляет не более 12 000 000 рублей ) . Так как нет смысла подавать заявку на Ипотеку в банк без суммы и выбора программы ипотеки, как указал это в своем ответе банк. Прошу принять меры по исправлению моей кредитной истории.',
    'Прошу оказать содействие и принять меры по факту правомерности ответа банка по вопросу несоответствия номера кредитного договора банка номеру кредитного договора из выписки БКИ. Так же по данному вопросу получен ответ, который прилагаю. Я желаю полностью досрочно погасить автокредит при условии официального подтверждения в установленной форме того, что оплачиваю именно свой кредит, в котором соответствует номер кредитного договора и УИД, и(или) иное официальное подтверждение в установленной форме о том, что номер договора не имеет значение, имеет значение только УИД.',
    'По обстоятельствам нарушения моих законных прав в связи с невыплатой в мой адрес денежных средств по накопительному счету открытому в банке в рамках "приветственного бонуса" мною было сформировано обращение в банк 04.10.2024. 30.10.2024 мне пришел ответ от банка, согласно которому накопительный счет открывается в банковском приложении клиентом банка, однако я не был проинформирован сотрудниками банка о необходимости каких либо действий с моей стороны для открытия "накопительного счета" помимо факта первичного внесения денежных средств на счет в банк. За весь период когда мои денежные средства находились на расчетном счете в банке - мне в формате СМС многократно приходили различные банковские предложения (оформить кредитную карты, осуществить коммунальные платежи и т.д.) однако ни разу мне не сообщили, что по факту моего первоначального обращение не осуществлены действия по начислению мне положенных законом процентов как следствие сотрудник банка к которому я изначально обратился за консультацией и содействием в оказании услуги не выполнены обязательства и не оказана мне услуга. Предполагаю, что кредитное учреждение умышленно игнорирует существенные обстоятельства и осуществляет "формальные отписки" с целью затягивания всех процессов и невозврата мне моих законных средств, а именно % начисленных банком за пользование моими денежными средствами.',
    'Здравствуйте ! 26.09. 2024 г мной был отрыт накопительный счет в банке. В условиях были заманчивые проценты на первые 2 календарных месяца - 20%. С ежемесячным начислением процентов на остаток. 26 ноября- в конце срока зашла в личный кабинет и увидела, что процентная ставка по счету 11%, вместо 20. Мною было составлено обращение в банк, с запросом на перерасчет процентов в соотвествии с условиями. От банка получен отказ. Банк считает, что 5 дней сентября- это первый календарный месяц после открытия и октябрь - второй. Ноябрь же начислялись пониженные проценты на накопительный счет. Считаю, что период короче, указанного в условиях при открытии счета и за ноябрь месяц до 26 числа- входит в 2 календарных месяца с даты открытия счета. Прошу рассмотреть мое обращение, Принять меры в отношении банка - за введение в заблуждение клиентов и доначислить проценты за полные 2 календарных месяца нахождения денежных средств на накопительном счете.',
    'Прошу в связи с систематическими мошенническими схемами вкладов в банке, наказать руководителей банка и отстранить их от работы! Банк по моему вкладу пользовался моими вкладами полный период пролонгации в период с 01 по 30 октября , не заплатив ни копеечки за мои деньги, то есть просто обманул меня пред пенсионера. Работники банка врали мне, и не начислил проценты =около 900 рублей на 60.000 рублей (18% годовых, позор просто) вклада, это обман чистой воды, они врут при приеме денег, сволочи, в итоге. ПРОШУ НЕ ПЕРЕДАВАТЬ МОЕ ЗАЯВЛЕНИЕ В БАНК а ПРОШУ КОМПЕТЕНТНЫХ СПЕЦИАЛИСТОВ ЦЕНТРАЛЬНОГО БАНКА РФ ПРОВЕСТИ КОМПЛЕКСНОЕ РАССЛЕДОВАНИЕ ПО ОБМАНУ ГРАЖДАН в БАНКЕ!, с Мошенниками с БАНКА!'
]
'''

queries = [
    'Здравствуйте. 07.03.2022 был открыт вклад в долларах США до 03.09.2022. 03.09.2022 было получено 10 000 долларов наличными и открыт новый вклад до 09.03.2023, затем открыт новый вклад до 15.06.2023, далее открыт вклад до 14.08.2025. Т.е. изначально валюта поступила в банк до 09.09.2022. Вопрос: по какому курсу в рубля (ЦБ или внутреннему курсу) банк должен вернуть средства с валютного вклада?',
    'Здравствуйте! Разъясните, пожалуйста, позицию ЦБ РФ по вопросу возникших юридических проблем с открытыми валютными вкладами. Вопрос 1 :  могу ли я подарить валютный вклад, открытый до 09.03.22 и валюту, находящуюся на счете до 01.09.22 своему совершеннолетнему ребенку и каким образом ребенок сможет воспользоваться вкладом : получить разрешенные 10 тысяч долларов? или вклад и валюта счета обязательно конвертируются в рубли? Может ли он продолжить хранить валюту на счете? Вопрос 2 : в связи с необходимостью оформления наследства : каким образом наследуется валютный вклад , открытый до 09.03.2022 г.Какие действия можно с ним производить в случае одного наследника и в случае двух наследников? К сожалению, в РСХБ не смогли однозначно ответить на данные вопросы.',
    'Добрый день. Прошу дать пояснения по следующей ситуации. В тексте указа президента России «О дополнительных временных мерах экономического характера по обеспечению финансовой стабильности Российской Федерации» от 1 марта 2022 (далее – Указ) говориться о том, что я как физическое лицо не могу пополнять свои иностранные счета в валюте, но тем временем Вы (ЦБ РФ) увеличили лимит на переводы из РФ на счета, открытые за границей с 10 000 USD до 50 000 USD. Так же я имею права пользоваться услугами по переводу денежных средств без открытия банковского счета, предоставленными российскими финансовыми организациями. В виду трудностей интерпретации большого количества противоречий информации прошу дать пояснения по следующим вопросам: 1. Какие именно операции пополнения попадают под действие данного указа? 2. Могу ли я осуществлять переводы в валюте с использованием системы SWIFT со своих счетов открытых в Российской федерации и/или заграничных счетов на счета, открытые за границей? (Если можно, то в какие страны переводы под запретом?) 3. Могу ли я принимать переводы, осуществляемые без открытия банковского счета напрямую на счета в иностранных банках в случаях: а) если отправителем являюсь я сам б) отправитель – другое физическое лицо. Есть ли у этого метода ограничения, которые необходимо учитывать? 4. Распространяются ли ограниченные данного указа и/или других законодательных актов на валютных резидентов, которые более 183 дней находятся за пределами РФ в отношении совершения переводов со своих/на свои иностранные счета в валюте и совершения иных действия по этим счетам, включая пополнение иностранных счетов различными способами, внесение надличных на свои иностранные счета, переводы/пополнения третьими лицами своих иностранных счетов и иные возможные операции. 5. Каким правовым статусом обладают указы Президента РФ в части финансового и валютного регулирования? Как называются нарушения подобных указов? Насколько в отношении вышеобозначенного Указа и других указов президента РФ распространяется КоАП РФ Статья 15.25. (Нарушение валютного законодательства Российской Федерации и актов органов валютного регулировани и другие меры воздействия)? В статье 5 173-ФЗ «О валютном регулировании и валютном контроле» говорится о том, что «Органами валютного регулирования в Российской федерации являются Центральный банк Российской Федерации и Правительство Российской Федерации.». Как видно, президент РФ в этот список не входит, а значит за нарушения Указа не могут применяться санкции, применяемые за нарушение валютного законодательства.',
    'Добрый день! Интересует вопрос касательно использования денежных счетов со счета типа С, открытого в банке нерезиденту 1) согласно решению СД ЦБ с банковского счета типа «С» могут списываться денежные средства в целях переводов на покупку облигаций федерального займа, размещаемых Минфином России на аукционах, а также при обращении таких облигаций. Однако в настоящее время банки отказываются осуществлять данные операции в связи с тем, что нет порядка использования денежных средств. Когда планируется опубликовать данный порядок, чтобы осуществить разрешенные операции со счета типа С? 2) Предусмотрена ли выдача разрешений нерезидентам на использование денежных средств, хранящихся на счетах типа С, на иные не указанные в решении СД цели (например, на конвертацию в доллары США)?'
]

doc_K1 = 30 #параметр задан в реализации RAG в модуле rag.py
doc_K2 = 5 #параметр задан в реализации RAG в модуле rag.py

rows = []

for i, qst in enumerate(queries):
    search_results = search_base.search_doc(qst, doc_K1, doc_base)
    rerank_results = search_base.rerank_doc(qst, search_results, doc_K2)
    #search_results = search_base.search_doc_late(qst, doc_base)

    for chunk in rerank_results:
        doc_name = chunk.metadata.get('source', 'unknown')

        #pattern = r"<#####>\s*(.*?)\s*(?=\{'source':)"
        #match = re.search(pattern, str(chunk), re.DOTALL)
        #law_text = match.group(1).strip() if match else ""
        pattern = r'\{.*?\}'
        match = re.search(pattern, str(chunk), re.DOTALL)
        cleared_chunk = re.sub(pattern, '', str(chunk))

        rows.append({
            "Номер вопроса": i+1,
            "Вопрос": qst,
            "Название НПА": doc_name,
            "Выделенный текст из закона": cleared_chunk,
            "Релевантность текста вопросу (1 - да/0 - нет)": ""
        })

df = pd.DataFrame(rows, columns=["Номер вопроса", "Вопрос", "Название НПА", "Выделенный текст из закона", "Релевантность текста вопросу (1 - да/0 - нет)"])

df.head()


In [ ]:
df.to_excel("e5_faiss_late_chunk_split.xlsx", index=False)

In [10]:
class LLM():
    """
        класс LLM
    """
    def __init__(self):
        self.mod_name = 'Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24'
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.llm = AutoModelForCausalLM.from_pretrained(
            self.mod_name,
            torch_dtype="auto",
            device_map="auto",
            output_hidden_states=False,
            do_sample=True,
            temperature=0.2  
        )

        self.tokenizer = AutoTokenizer.from_pretrained(self.mod_name)


    def invoke(self, prmt):
        search_config = Config()
        sys_prompt = search_config.system_prompt
        try:
            messages = [
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": prmt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                truncation=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)
            full_generated_ids = self.llm.generate(
                model_inputs.input_ids,
                attention_mask=model_inputs.attention_mask,
                max_new_tokens=1024,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
            new_generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, full_generated_ids)]
            res0 = self.tokenizer.batch_decode(new_generated_ids, skip_special_tokens=True)[0]
        except Exception as ex:
            print(ex)
            res0 = None
        return res0

    def change_temp(self, temp):
        # self.llm.model.config.temperature = temp
        self.llm.generation_config.temperature = temp


In [11]:
my_llm = LLM()

Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.30it/s]


In [ ]:
import pandas as pd

queries = [
    'Здравствуйте. 07.03.2022 был открыт вклад в долларах США до 03.09.2022. 03.09.2022 было получено 10 000 долларов наличными и открыт новый вклад до 09.03.2023, затем открыт новый вклад до 15.06.2023, далее открыт вклад до 14.08.2025. Т.е. изначально валюта поступила в банк до 09.09.2022. Вопрос: по какому курсу в рубля (ЦБ или внутреннему курсу) банк должен вернуть средства с валютного вклада?',
    'Здравствуйте! Разъясните, пожалуйста, позицию ЦБ РФ по вопросу возникших юридических проблем с открытыми валютными вкладами. Вопрос 1 :  могу ли я подарить валютный вклад, открытый до 09.03.22 и валюту, находящуюся на счете до 01.09.22 своему совершеннолетнему ребенку и каким образом ребенок сможет воспользоваться вкладом : получить разрешенные 10 тысяч долларов? или вклад и валюта счета обязательно конвертируются в рубли? Может ли он продолжить хранить валюту на счете? Вопрос 2 : в связи с необходимостью оформления наследства : каким образом наследуется валютный вклад , открытый до 09.03.2022 г.Какие действия можно с ним производить в случае одного наследника и в случае двух наследников? К сожалению, в РСХБ не смогли однозначно ответить на данные вопросы.',
    'Добрый день. Прошу дать пояснения по следующей ситуации. В тексте указа президента России «О дополнительных временных мерах экономического характера по обеспечению финансовой стабильности Российской Федерации» от 1 марта 2022 (далее – Указ) говориться о том, что я как физическое лицо не могу пополнять свои иностранные счета в валюте, но тем временем Вы (ЦБ РФ) увеличили лимит на переводы из РФ на счета, открытые за границей с 10 000 USD до 50 000 USD. Так же я имею права пользоваться услугами по переводу денежных средств без открытия банковского счета, предоставленными российскими финансовыми организациями. В виду трудностей интерпретации большого количества противоречий информации прошу дать пояснения по следующим вопросам: 1. Какие именно операции пополнения попадают под действие данного указа? 2. Могу ли я осуществлять переводы в валюте с использованием системы SWIFT со своих счетов открытых в Российской федерации и/или заграничных счетов на счета, открытые за границей? (Если можно, то в какие страны переводы под запретом?) 3. Могу ли я принимать переводы, осуществляемые без открытия банковского счета напрямую на счета в иностранных банках в случаях: а) если отправителем являюсь я сам б) отправитель – другое физическое лицо. Есть ли у этого метода ограничения, которые необходимо учитывать? 4. Распространяются ли ограниченные данного указа и/или других законодательных актов на валютных резидентов, которые более 183 дней находятся за пределами РФ в отношении совершения переводов со своих/на свои иностранные счета в валюте и совершения иных действия по этим счетам, включая пополнение иностранных счетов различными способами, внесение надличных на свои иностранные счета, переводы/пополнения третьими лицами своих иностранных счетов и иные возможные операции. 5. Каким правовым статусом обладают указы Президента РФ в части финансового и валютного регулирования? Как называются нарушения подобных указов? Насколько в отношении вышеобозначенного Указа и других указов президента РФ распространяется КоАП РФ Статья 15.25. (Нарушение валютного законодательства Российской Федерации и актов органов валютного регулировани и другие меры воздействия)? В статье 5 173-ФЗ «О валютном регулировании и валютном контроле» говорится о том, что «Органами валютного регулирования в Российской федерации являются Центральный банк Российской Федерации и Правительство Российской Федерации.». Как видно, президент РФ в этот список не входит, а значит за нарушения Указа не могут применяться санкции, применяемые за нарушение валютного законодательства.',
    'Добрый день! Интересует вопрос касательно использования денежных счетов со счета типа С, открытого в банке нерезиденту 1) согласно решению СД ЦБ с банковского счета типа «С» могут списываться денежные средства в целях переводов на покупку облигаций федерального займа, размещаемых Минфином России на аукционах, а также при обращении таких облигаций. Однако в настоящее время банки отказываются осуществлять данные операции в связи с тем, что нет порядка использования денежных средств. Когда планируется опубликовать данный порядок, чтобы осуществить разрешенные операции со счета типа С? 2) Предусмотрена ли выдача разрешений нерезидентам на использование денежных средств, хранящихся на счетах типа С, на иные не указанные в решении СД цели (например, на конвертацию в доллары США)?'
]

results = []

for query in queries:
    llm_res, qq3, res3, exp_docs, prompt = search_base.get_response(my_llm, query, doc_base)
    
    results.append({
        "Запрос": query,
        "Промпт": prompt,
        "Ответ RAG": llm_res
    })

df = pd.DataFrame(results)
df.to_excel("results_generation_e5_faiss.xlsx", index=False)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1532: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [14]:
df.head()

,Запрос,Промпт,Ответ RAG
0,Здравствуйте. 07.03.2022 был открыт вклад в до...,"Детально и подробно ответь на вопрос, использу...","В соответствии с предоставленными данными, кли..."
1,"Здравствуйте! Разъясните, пожалуйста, позицию ...","Детально и подробно ответь на вопрос, использу...",### Ответ на вопрос 1:\n\nСогласно предоставле...
2,Добрый день. Прошу дать пояснения по следующей...,"Детально и подробно ответь на вопрос, использу...",### Ответ на вопросы клиента\n\n1. **Какие име...
3,Добрый день! Интересует вопрос касательно испо...,"Детально и подробно ответь на вопрос, использу...",Добрый день! В соответствии с доступной мне ин...
